In [ ]:
import torch
import torchvision
import torchvision.datasets
import torchvision.transforms
import torch.utils.data
import torch.nn
import os
from matplotlib import pyplot as plot

In [ ]:
# Загрузка набора данных MNIST

dir_name = os.getcwd() # скачивание данных выполняется в текущую директорию
batch_size = 100 # размер обрабатываемой пачки данных

# Функция для демонстрации примеров изображений
def show_images(images, title):
    num_showed_imgs_x = 10
    num_showed_imgs_y = 10
    
    figsize = (5, 5)
    fig, axes = plot.subplots(num_showed_imgs_y, num_showed_imgs_x, figsize = figsize)
    fig.suptitle(title)
    plot.setp(plot.gcf().get_axes(), xticks = [], yticks = [])
    for i, ax in enumerate(axes.flat):
        # images[i][0] - многомерный массив типа torch.Tensor
        # images[i][1] - число типа numpy.int32
        img = images[i][0].numpy().transpose(1, 2, 0).squeeze(axis = 2)
        ax.imshow(img, cmap = 'gray')

# Чтение тренировочной и тестовой выборок набора данных MNIST.
# Данные представляются в виде пар (tuple), где первый элемент - 
# изображение в формате PIL.Image.Image, а второй - целочисленная
# метка класса.Параметр transform обеспечивает преобразование
# изображений в формат torch.Tensor для последующей работы
train_dataset = torchvision.datasets.MNIST(root = dir_name, train = True, download = True,
                                           transform = torchvision.transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root = dir_name, train = False, download = True,
                                          transform = torchvision.transforms.ToTensor())

# Логирование информации о загруженных данных
print('Number of train samples: {}'.format(len(train_dataset)))
show_images(train_dataset, 'Train samples')

print('Number of test samples: {}'.format(len(test_dataset)))
show_images(test_dataset, 'Test samples')

# Создание объектов для последовательной загрузки пачек
# из тренировочной и тестовой выборок
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size,
                                                shuffle = True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size,
                                               shuffle = False)

In [ ]:
# Создание модели, соответствующей логистической регрессии

# Создание класса сети, соответствующей логистической регрессии
class ConvolutionalNeuralNetwork(torch.nn.Module):
    # Объявление конструктора
    def __init__(self):
        super(ConvolutionalNeuralNetwork, self).__init__()
        # Свертка. Размер результирующей карты признаков:
        # 10 x ((28-3)/1+1) x ((28-3)/1+1) = 10 x 26 x 26
        self.conv = torch.nn.Conv2d(in_channels = 1, out_channels = 10, 
                                    kernel_size = 3, stride = 1)
        # Функция активации. Размер результирующей карты признаков:
        # 10 x 26 x 26
        self.activation = torch.nn.ReLU()
        # Пространственное объединение по максимуму. Размер результирующей карты признаков:
        # 10 x ((26-2)/1+1) x ((26-2)/1+1) = 10 x 25 x 25
        self.pooling = torch.nn.MaxPool2d(kernel_size = 2, stride = 1)
        # Создание полносвязного слоя (softmax включается в вычисление кросс-энтропии)
        self.linear = torch.nn.Linear(10 * 25 * 25, 10)
    # Переопределение метода, вызываемого в процессе прямого прохода
    def forward(self, x):
        # Свертка
        out = self.conv(x)
        # Функция активации
        out = self.activation(out)
        # Пространственное объединение
        out = self.pooling(out)
        # Изменение формата хранения тензора из (B, C, W, H) в (B, C*W*H)
        out = out.view(out.size(0), -1)
        # Полносвязный слой
        out = self.linear(out)
        return out

# Создание объекта разработанного класса
cnn_model = ConvolutionalNeuralNetwork()

# Логирование информации о параметрах модели
print('Parameters of convolutions:\n1. Kernels = {}\n2. Biases = {}'.
      format(cnn_model.conv.weight.shape, cnn_model.conv.bias.shape))
print('Parameters of fully-connected layer:\n1. Weight matrix = {}\n2. Biases = {}'.
      format(cnn_model.linear.weight.shape, cnn_model.linear.bias.shape))

In [ ]:
# Обучение построенной модели на CPU

learning_rate = 0.1 # скорость обучения
num_epochs = 5 # количество эпох

# Выбор устройства для вычислений
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn_model.to(device)
# Функция ошибки на этапе обучения
loss_function = torch.nn.CrossEntropyLoss()
# Метод оптимизации для обучения параметров
optimizer = torch.optim.SGD(cnn_model.parameters(), lr = learning_rate)

# Функция вычисления точности top-1
def get_accuracy(data_loader, model):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader: # проход по всем данным
            # Получение выхода сети на входной пачке изображений
            images = images.requires_grad_().to(device)
            labels = labels.to(device)
            outputs = model(images)
            # Выбор предсказанных меток с максимальной достоверностью.
            # outputs.data - объект типа torch.tensor, двумерный тензор, массив
            # векторов достоверности принадлежности каждому из 10 допустимых классов
            # (размерность 0 - номер изображения в пачке, размерность 1 - номер класса);
            # predicted - объект типа torch.tensor (одномерный тензор меток классов).
            # Выбор максимальных значений выполняется по первой размерности
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0) # количество изображений, совпадает с batch_size
            tp += (predicted == labels).sum() # определение количества корректных совпадений
    return tp / n

for epoch in range(num_epochs):  # проход по эпохам
    for i, (images, labels) in enumerate(train_data_loader): # получение пачки тренировочных данных
        # Загрузка данных на устройство
        images = images.requires_grad_().to(device)
        labels = labels.to(device)
        # Прямой проход
        outputs = cnn_model(images) # вычисление выхода сети
        loss = loss_function(outputs, labels) # вычисление функции ошибки, loss.item() дает значение
        # Обратный проход
        optimizer.zero_grad() # обнуление всех вычисляемых градиентов
        loss.backward() # вычисление градиента функции ошибки
        optimizer.step() # обновление параметров модели
    
    # Логирование метрики качества на тренировочных данных по завершении эпохи
    print('Epoch[{}]: accuracy = {}'.
          format(epoch, get_accuracy(train_data_loader, cnn_model)))

In [ ]:
# Тестирование обученной модели

# Логирование метрики качества на тренировочных данных
print('Test accuracy: {}'.format(get_accuracy(test_data_loader, cnn_model)))